In [ ]:
# importer les bibliothèques nécessaires

import datetime
import streamlit as st
import pandas_datareader as pdr
import cufflinks as cf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import yfinance as yf
import requests, json
import seaborn as sb
import plotly.express as px
import plotly.graph_objects as go
import plotly.offline as py
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import math
from sklearn.metrics import mean_squared_error, mean_absolute_error, explained_variance_score, r2_score 
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.layers import LSTM
from itertools import cycle

APP_NAME = "Cryptocurrency Candlestick Dashboard"

# Page Configuration
st.set_page_config(
    page_title=APP_NAME,
    layout="wide",
    initial_sidebar_state="expanded",
)

# Add some markdown
#st.sidebar.markdown("Enjoy vizualisation!!!")


# Add app title
st.sidebar.title(APP_NAME)

# List of tickers
TICKERS = ['BTC-USD','ETH-USD','BNB-USD','XRP-USD','ADA-USD',
           'SOL-USD','MATIC-USD','DOT-USD','TRX-USD','UNI-USD',
           'ATOM-USD','LINK-USD','ETC-USD','BCH-USD','AAVE-USD']

# Select ticker
ticker = st.sidebar.selectbox('Select ticker', sorted(TICKERS), index=0)

# Set start and end point to fetch data
start_date = st.sidebar.date_input('Start date', datetime.datetime(2018, 1, 1))
end_date = st.sidebar.date_input('End date', datetime.datetime.now().date())

# Fetch the data for specified ticker e.g. AAPL from yahoo finance
df_ticker = pdr.DataReader(ticker, 'yahoo', start_date, end_date)

st.header(f'{ticker} Price evolution')

if st.checkbox('Show raw data'):
    st.subheader('Raw data')
    st.write(df_ticker)
    
    
# Interactive data visualizations using cufflinks
# Create candlestick chart 
qf = cf.QuantFig(df_ticker, legend='top', name=ticker,down_color='red',up_color='green')


# Technical Analysis Studies can be added on demand
# Add Relative Strength Indicator (RSI) study to QuantFigure.studies
qf.add_rsi(periods=20, color='black')
# Add MACD
qf.add_macd(color = ['red', 'black'])
# Add 'volume' study to QuantFigure.studies
#qf.add_volume()



qf.add_annotations([
                               dict(text = "RSI",
                                    font = dict(size = 15, color ='white'),
                                    showarrow=False,
                                    x = 0.5, y = 0.38,
                                    xref = 'paper', yref = "paper",
                                    align = "center",textangle=0),
                              
                               dict(text = "MACD",
                                    font = dict(size = 15, color ='white'),
                                    showarrow=False,
                                    x = 0.5, y = 0.14,
                                    xref = 'paper', yref = "paper",
                                    align = "center",textangle=0),
                               ])

fig = qf.iplot(asFigure=True, dimensions=(900, 1000))

# Render plot using plotly_chart
st.plotly_chart(fig)

#########################################################Prediction 

st.header(f'{ticker} Price Prediction using Deep Learning')



def DataPrep(data):
    # reseved method in python classes (Constructor)
    # We are taking only the Close prices for predicting 
    data = data.filter(['Close'])
    dataset = data.values
    # We take 70% of the data for training and 30% for testing 
    perc_train = 70 
    training_data_len = int(np.ceil( len(dataset) * (perc_train/100)))
    
    # We are scaling the open prices to the range(0,1)
    scaler = MinMaxScaler(feature_range=(0,1))
    scaled_data = scaler.fit_transform(np.array(dataset).reshape(-1,1))
    # Taking the first 70% of the dataset for training 
    train_data = scaled_data[0:int(training_data_len), :]
    test_data = scaled_data[int(training_data_len):len(dataset), :]
    # Split the data into self.X_train and self.y_train data sets
    X_train = []
    y_train = []
    
    # We are taking predicting the open price of a given day based on the trend in the previous 60 days
    def create_dataset(dataset, time_step=1):
        dataX, dataY = [], []
        for i in range(len(dataset)-time_step-1):
            a = dataset[i:(i+time_step), 0]   #i=0, 0,1,2,3-----99   100 
            dataX.append(a)
            dataY.append(dataset[i + time_step, 0])
        return np.array(dataX), np.array(dataY)
    
    X_train,y_train = create_dataset(train_data,time_step=15)
    X_test,y_test = create_dataset(test_data,time_step=15)
    return X_train,y_train,X_test,y_test,scaled_data
                
        
def LSTM_model(X_train,y_train,X_test,y_test,scaled_data):
    LSTM_model=Sequential()
    LSTM_model.add(LSTM(100,input_shape=(None,1), return_sequences=True,activation="relu"))
    LSTM_model.add(LSTM(100, return_sequences=True))
    LSTM_model.add(LSTM(100, return_sequences=False))
    LSTM_model.add(Dense(25))
    LSTM_model.add(Dense(1))
    LSTM_model.compile(loss="mean_squared_error",optimizer="adam")
    # Train the model
    LSTM_model.fit(X_train,y_train, epochs= 20,
                   validation_data=(X_test,y_test),batch_size=32,verbose=True)
    
    #Get the models predicted price values 
    train_predict=LSTM_model.predict(X_train)
    predictions=LSTM_model.predict(X_test)
    return train_predict,predictions,scaled_data

#scaler = MinMaxScaler(feature_range=(0,1))
#scaled_data = scaler.fit_transform(np.array(df_ticker).reshape(-1,1))


X_train,y_train,X_test,y_test,scaled_data=DataPrep(df_ticker)    
train_predict,predictions,scaled_data=LSTM_model(X_train,y_train,X_test,y_test,scaled_data)    
#train_predict= scaler.inverse_transform(train_predict)
#predictions=scaler.inverse_transform(predictions)



st.title("Long Short-Term Memory (LSTM)")        
# We need to inverse transform the scaled data to compare it with our unscaled y_test data
#predictions = scaler.inverse_transform(predictions)
st.text("R2 SCORE")
st.text(r2_score(y_test, predictions))
#st.text("MSLE")
#st.text(mean_squared_log_error(y_test, predictions))
plt.plot(predictions)
plt.plot(y_test)
plt.legend(["Predicted","Observed"])
#plt.xticks(range(0,len(y_test),50),testd,rotation=45)
plt.xlabel('Date',fontsize=18)
plt.ylabel('Price',fontsize=18)
plt.title("LSTM")
st.pyplot(plt)


# shift train predictions for plotting
time_step=15
look_back=time_step
trainPredictPlot = np.empty_like(scaled_data)
trainPredictPlot[:, :] = np.nan
trainPredictPlot[look_back:len(train_predict)+look_back, :] = train_predict

# shift test predictions for plotting
testPredictPlot = np.empty_like(scaled_data)
testPredictPlot[:, :] = np.nan
testPredictPlot[len(train_predict)+(look_back*2)+1:len(scaled_data)-1, :] = predictions

names = cycle(['Original close price','Train predicted close price','Test predicted close price'])


plotdf = pd.DataFrame({'date': df_ticker.index,
                       'original_close': df_ticker['Close'],
                      'train_predicted_close': trainPredictPlot.reshape(1,-1)[0].tolist(),
                      'test_predicted_close': testPredictPlot.reshape(1,-1)[0].tolist()})

fig = px.line(plotdf,x=plotdf['date'], y=[plotdf['original_close'],plotdf['train_predicted_close'],
                                          plotdf['test_predicted_close']],
              labels={'value':'Close price','date': 'Date'})
fig.update_layout(title_text='Prediction avec LSTM',
                  plot_bgcolor='white', font_size=15, font_color='black', legend_title_text='Close Price')
fig.for_each_trace(lambda t:  t.update(name = next(names)))

fig.update_xaxes(showgrid=False)
fig.update_yaxes(showgrid=False)
fig.show()
st.plotly_chart(fig)


